# Lab program 7: Fake news prediction

In [3]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Disha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df=pd.read_csv('fake_news_train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [8]:
# Missing values handling
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
# replacing the null values with empty string
df = df.fillna('')

In [11]:
# merging the author name and news title
df['content'] = df['author']+' '+df['title']

In [12]:
# dataset after combining author and content
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


# Stemming: 

In [19]:
port = PorterStemmer()
def stemming(content):  # create a function by name stemming with some input
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # replace all special symbols with space except A-Z and a-z
    stemmed_content = stemmed_content.lower() # convert alphabets to lower case
    stemmed_content = stemmed_content.split() # converted to list
    stemmed_content = [port.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # take the words which are not stopwords
    stemmed_content = ' '.join(stemmed_content) # join all the words after
    return stemmed_content

In [20]:
df['content'] = df['content'].apply(stemming) # apply the function to the content column of the data set

In [21]:
df['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [27]:
#feature set and target variable
x = df['content'].values
y = df['label'].values

In [28]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [29]:
# Split into training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=2) # data will be splitted stratified way

In [31]:
# Training a model and
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [37]:
# accuracy score
predictions = model.predict(x_test)
score = accuracy_score(predictions, y_test)
score

0.9790865384615385

# Decision tree classifier 

In [41]:
from sklearn.tree import DecisionTreeClassifier
classifier=DecisionTreeClassifier()

In [42]:
# Training a model
classifier.fit(x_train, y_train)

DecisionTreeClassifier()

In [43]:
# Predictions
# accuracy score
predictions = classifier.predict(x_test)
score = accuracy_score(predictions, y_test)
score

0.9918269230769231

In [44]:
# Predictions with data pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


In [55]:
# 1st classification algorithm  (Decision Tree)
pipeline_dec =Pipeline([('scalar1',StandardScaler(with_mean=False)),
                       ('decision_tree',DecisionTreeClassifier())])

In [56]:
# 2nd classification algorithm  (KNN)
pipeline_knn =Pipeline([('scalar2',StandardScaler(with_mean=False)),
                        ('knn_classifier',KNeighborsClassifier())])

In [57]:
# 3rd classification algorithm  (Logistic Regression)
pipeline_log =Pipeline([('scalar3',StandardScaler(with_mean=False)),
                        ('logistic_regression',LogisticRegression())])

In [58]:
# Create the pipeline
pipelines=[pipeline_dec, pipeline_knn,pipeline_log]

In [59]:
# initialize the variables
best_accuracy=0.0
best_classifier=0
best_pipeline=""

In [60]:
# Fit the pipeline
for pipe in pipelines:
    pipe.fit(x_train, y_train)

In [64]:
for i, model in enumerate(pipelines):
    score=model.score(x_test,y_test)
    print(score)

0.9920673076923077
0.5548076923076923
0.9834134615384615


In [67]:
# Pickle the model file for deployment
import pickle # to do the pickling of the regression model
pickle.dump(classifier, open('decision.pkl','wb'))


In [68]:
# load the pickle
pickled_model=pickle.load(open('decision.pkl','rb'))

In [75]:
pickled_model.predict((x_test))[:20]

array([1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0],
      dtype=int64)

In [77]:
predictions[:20]

array([1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0],
      dtype=int64)